In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/kerasDO/master/HR_comma_sep.csv")

In [2]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
feats = ['department','salary']
df_final = pd.get_dummies(df,columns=feats,drop_first=True)

In [4]:
df_final.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,1,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,1,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,1,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0


In [5]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 19 columns):
satisfaction_level        14999 non-null float64
last_evaluation           14999 non-null float64
number_project            14999 non-null int64
average_montly_hours      14999 non-null int64
time_spend_company        14999 non-null int64
Work_accident             14999 non-null int64
left                      14999 non-null int64
promotion_last_5years     14999 non-null int64
department_RandD          14999 non-null uint8
department_accounting     14999 non-null uint8
department_hr             14999 non-null uint8
department_management     14999 non-null uint8
department_marketing      14999 non-null uint8
department_product_mng    14999 non-null uint8
department_sales          14999 non-null uint8
department_support        14999 non-null uint8
department_technical      14999 non-null uint8
salary_low                14999 non-null uint8
salary_medium             14999 non-null 

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df_final.drop(['left'],axis=1).values
y = df_final['left'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [9]:
X_test[0]

array([   0.92,    0.64,    4.  ,  190.  ,   10.  ,    1.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    1.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    1.  ,    0.  ])

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [12]:
classifier = Sequential()

In [13]:
classifier.add(Dense(9, kernel_initializer = "uniform",activation = "relu", input_dim=18))

In [14]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [15]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [16]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)

Epoch 1/1
10499/10499 [==============================] - 2s 181us/step - loss: 0.4219 - acc: 0.7985


In [17]:
y_pred = classifier.predict(X_test)

In [18]:
y_pred = (y_pred > 0.5)

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3333,  108],
       [ 605,  454]])

In [20]:
cm


array([[3333,  108],
       [ 605,  454]])

In [21]:
new_pred = classifier.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.]])))

In [22]:
new_pred = (new_pred > 0.5)
new_pred

array([[False]], dtype=bool)

In [23]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [24]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [25]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)

In [26]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 2,n_jobs = -1)

In [27]:
mean = accuracies.mean()

In [28]:
variance = accuracies.var()

In [29]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [38]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [39]:
classifier = KerasClassifier(build_fn = make_classifier)

In [40]:
params = {
    'batch_size':[20,35],
    'nb_epoch':[5,10],
    'optimizer':['adam','rmsprop']
}

In [41]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=10)

In [42]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/1
9449/9449 [==============================] - 1s 73us/step - loss: 0.5153 - acc: 0.7586
Epoch 1/1
9449/9449 [==============================] - 1s 72us/step - loss: 0.5020 - acc: 0.7673
Epoch 1/1
9449/9449 [==============================] - 1s 67us/step - loss: 0.5009 - acc: 0.7671
Epoch 1/1
9449/9449 [==============================] - 1s 70us/step - loss: 0.5097 - acc: 0.7590
Epoch 1/1
9449/9449 [==============================] - 1s 73us/step - loss: 0.5014 - acc: 0.7732
Epoch 1/1
9449/9449 [==============================] - 1s 74us/step - loss: 0.4997 - acc: 0.7712
Epoch 1/1
9449/9449 [==============================] - 1s 78us/step - loss: 0.5087 - acc: 0.7919
Epoch 1/1
9449/9449 [==============================] - 1s 90us/step - loss: 0.5000 - acc: 0.7787
Epoch 1/1
9449/9449 [==============================] - 1s 91us/step - loss: 0.5103 - acc: 0.7707
Epoch 1/1
9450/9450 [==============================] - 1s 89us/step - loss: 0.5064 - acc: 0.7643
Epoch 1/1
9449/9449 [=========

KeyboardInterrupt: 

In [ ]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_